In [ ]:
!pip install --pre pycaret

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from math import sqrt
import os

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from tqdm.notebook import tqdm
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.linear_model import Lasso,Ridge
from sklearn.pipeline import Pipeline

import lightgbm as lgbm

### 데이터 로딩

In [ ]:
### 일자를 기준으로 train test 나누기 이전의 데이터 불러오는 코드.
# test_path = "/content/drive/MyDrive/함께하조/기계학습과 딥러닝/data/kaggle_data/test_eda.csv"
# train_path = "/content/drive/MyDrive/함께하조/기계학습과 딥러닝/data/kaggle_data/train_eda.csv"

# test = pd.read_csv(test_path)
# train = pd.read_csv(train_path)

# test.drop("Unnamed: 0", axis=1, inplace=True)
# train.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
test_path = "/content/drive/MyDrive/함께하조/기계학습과 딥러닝/data/kaggle_data/test_eda.csv"
train_path = "/content/drive/MyDrive/함께하조/기계학습과 딥러닝/data/kaggle_data/train_eda.csv"

# ## 주피터용
# test_path = "test_eda.csv"
# train_path = "train_eda.csv"
# train_og = pd.read_csv('train.csv')
# test_og = pd.read_csv('test.csv')

test = pd.read_csv(test_path)
train = pd.read_csv(train_path)

test.drop("Unnamed: 0", axis=1, inplace=True)
train.drop("Unnamed: 0", axis=1, inplace=True)

#cat features
cat_col = ["season", "Year","weather", "Day of week","Month", "Hour","Day_info"]
for col in cat_col:
    train[col] = train[col].astype("category")
    test[col] = test[col].astype("category")

#train, valid split
days = list(range(1, 15))
train_d = train.loc[train['Day'].isin(days)]
test_d = train.loc[~(train['Day'].isin(days))]

#target, drop, y
target_col = "count"
drop_cols = ["datetime", "workingday", "holiday", "Day", "Year","sin_hour", "cos_hour", target_col]

####
# train = make_di(train)
# last_target = train['count']
# train_d = make_di(train_d)
# test_d = make_di(test_d)
# test = make_di(test)

# train_d.drop(columns = 'temp',inplace = True)
# test_d.drop(columns = 'temp',inplace = True)
# test.drop(columns = 'temp',inplace = True)
# train.drop(columns = 'temp',inplace = True)

cat_col = ["season", "weather", "Day of week", "Month", "Hour", "Day_info"]

## category 지원 안되는 전용
# train = pd.get_dummies(train, columns=cat_col,drop_first = True)
# test = pd.get_dummies(test, columns=cat_col,drop_first = True)
# test_d = pd.get_dummies(test_d, columns=cat_col,drop_first = True)
# train_d = pd.get_dummies(train_d, columns=cat_col,drop_first = True)

## catboost 전용
train_d[cat_col] = train_d[cat_col].astype("category")
test_d[cat_col] = test_d[cat_col].astype("category")
train[cat_col] = train[cat_col].astype("category")
test[cat_col] = test[cat_col].astype("category")


####
test = test.drop(columns = ["datetime", "workingday", "holiday", "Day", "Year","sin_hour", "cos_hour"],axis = 1)
train.drop(drop_cols,axis=1,inplace = True)
x_train, y_train = train_d.drop(drop_cols, axis=1), train_d[target_col]
x_test, y_test = test_d.drop(drop_cols, axis=1), test_d[target_col]

x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

x_train.head()

,season,weather,temp,humidity,windspeed,Day of week,Month,Hour,Day_info
0,1,Good,9.84,81,0.0,Saturday,1,0,Weekend
1,1,Good,9.02,80,0.0,Saturday,1,1,Weekend
2,1,Good,9.02,80,0.0,Saturday,1,2,Weekend
3,1,Good,9.84,75,0.0,Saturday,1,3,Weekend
4,1,Good,9.84,75,0.0,Saturday,1,4,Weekend


x_train -> 학습에 훈련될 데이터셋 y_train -> 학습훈련될 타겟<br>
x_test -> 학습된 모델 평가 셋(y_test) <br>
train -> 전체 학습된 모델에 사용되는것 

### Pycaret

In [ ]:
from pycaret.regression import setup, eda, compare_models

In [ ]:
# target_col = "count"
# drop_cols = ["datetime", "workingday", "holiday", "Year", "Day", "sin_hour", "cos_hour",target_col]

# X, y = train.drop(drop_cols, axis=1), train[target_col]
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2023)

# x_train.reset_index(drop=True, inplace=True)
# y_train.reset_index(drop=True, inplace=True)

# x_test.reset_index(drop=True, inplace=True)
# y_test.reset_index(drop=True, inplace=True)

# x_train.head()

In [ ]:
#Pycaret 사용하기 위한 전처리
pc_train = pd.concat([x_train, y_train],axis=1)
pc_test = pd.concat([x_test, y_test],axis=1)

In [ ]:
pc_train.head()

,season,weather,temp,humidity,windspeed,Day of week,Month,Hour,Day_info,count
0,1,Good,9.84,81,0.0,Saturday,1,0,Weekend,16
1,1,Good,9.02,80,0.0,Saturday,1,1,Weekend,40
2,1,Good,9.02,80,0.0,Saturday,1,2,Weekend,32
3,1,Good,9.84,75,0.0,Saturday,1,3,Weekend,13
4,1,Good,9.84,75,0.0,Saturday,1,4,Weekend,1


In [ ]:
print(x_train.shape)
print(x_test.shape)
print(pc_train.shape)
print(pc_test.shape)

(8026, 9)
(2860, 9)
(8026, 10)
(2860, 10)


In [ ]:
cols = set(pc_train.columns)
numeric_cols = set(["temp", "humidity", "windspeed"])
cat_cols = cols-numeric_cols
cat_cols.remove("count")

numeric_cols = list(numeric_cols)
cat_cols = list(cat_cols)

print(numeric_cols)
print(cat_cols)

['windspeed', 'humidity', 'temp']
['Hour', 'weather', 'Month', 'Day of week', 'season', 'Day_info']


In [ ]:
py_data = setup(
    data=pc_train, target="count",
    train_size = 0.75,
    numeric_features = numeric_cols,
    categorical_features = cat_cols,
    normalize=True, normalize_method="zscore",
    fold=5, session_id=2023,
    use_gpu = True
)

,Description,Value
0,Session id,2023
1,Target,count
2,Target type,Regression
3,Original data shape,"(8026, 10)"
4,Transformed data shape,"(8026, 57)"
5,Transformed train set shape,"(6019, 57)"
6,Transformed test set shape,"(2007, 57)"
7,Numeric features,3
8,Categorical features,6
9,Preprocess,True


In [ ]:
# from pycaret.regression import *

# xgb = create_model('xgboost')

In [ ]:
best_models = compare_models(sort="RMSLE",exclude=['et'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,51.5105,5763.5359,75.9003,0.8237,0.5821,0.7876,2.8600
knn,K Neighbors Regressor,63.9894,8803.1739,93.8037,0.7310,0.5825,0.7057,0.2340
lightgbm,Light Gradient Boosting Machine,48.5866,4867.9882,69.7649,0.8511,0.6119,0.8343,0.3080
dt,Decision Tree Regressor,67.4453,11274.0334,106.1251,0.6553,0.6919,0.7716,0.3120
gbr,Gradient Boosting Regressor,66.1186,8681.9741,93.1712,0.7345,0.7928,1.3787,0.8660
huber,Huber Regressor,77.6029,12408.6798,111.3715,0.6207,0.9442,2.2045,0.2960
llar,Lasso Least Angle Regression,79.4312,12152.6156,110.2221,0.6284,1.0001,2.4613,0.2620
lasso,Lasso Regression,79.4304,12152.5553,110.2218,0.6284,1.0002,2.4613,0.3380
br,Bayesian Ridge,79.5687,12141.0010,110.1697,0.6288,1.0029,2.5313,0.3360
ridge,Ridge Regression,79.5774,12141.6468,110.1723,0.6288,1.0042,2.5464,0.3280


Processing:   0%|          | 0/77 [00:00<?, ?it/s]